In [1]:
# setup notebook
import numpy as np
import plotly.graph_objects as go
from bs4 import BeautifulSoup
import json
import time
import sys
sys.path.append('../../assets/python/')
import diceEq as de
import playercharacters5e as pc5
import encounters5e as en5


METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False
SAVEDATA = False

XP_TARGETS = {
    'Easy': [25,50,75,125,250,300,350,450,550,600,800,1000,1100,1250,1400,1600,2000,2100,2400,2800],
    'Medium': [50,100,150,250,500,600,750,900,1100,1200,1600,2000,2200,2500,2800,3200,3900,4200,4900,5700],
    'Hard': [75,150,225,375,750,900,1100,1400,1600,1900,2400,3000,3400,3800,4300,4800,5900,6300,7300,8500],
    'Deadly': [100,200,400,500,1100,1400,1700,2100,2400,2800,3600,4500,5100,5700,6400,7200,8800,9500,10900,12700],
    'Daily': [300,600,1200,1700,3500,4000,5000,6000,7500,9000,10500,11500,13500,15000,18000,20000,25000,27000,30000,40000]
}

PC_CLASSES = {
    'Barbarian': {'color': '#E7623E', 'group': 'martial'},
    'Bard': {'color': '#AB6DAC', 'group': 'spellcaster'},
    'Cleric': {'color': '#91A1B2', 'group': 'spellcaster'},
    'Druid': {'color': '#7A853B', 'group': 'spellcaster'},
    'Fighter': {'color': '#7F513E', 'group': 'martial'},
    'Monk': {'color': '#51A5C5', 'group': 'martial'},
    'Paladin': {'color': '#B59E54', 'group': 'martial-spellcaster'},
    'Ranger': {'color': '#507F62', 'group': 'martial-spellcaster'},
    'Rogue': {'color': '#555752', 'group': 'martial'},
    'Sorcerer': {'color': '#992E2E', 'group': 'spellcaster'},
    'Warlock': {'color': '#7B469B', 'group': 'spellcaster'},
    'Wizard': {'color': '#2A50A1', 'group': 'spellcaster'}
}

ENCOUNTER_DIFFICULTIES = {
    'Easy':   {
        'XP': [25,50,75,125,250,300,350,450,550,600,800,1000,1100,1250,1400,1600,2000,2100,2400,2800], 
        'color': '#1F77B4'
    },
    'Medium': {
        'XP': [50,100,150,250,500,600,750,900,1100,1200,1600,2000,2200,2500,2800,3200,3900,4200,4900,5700], 
        'color': '#FF7F0E'
    },
    'Hard':   {
        'XP': [75,150,225,375,750,900,1100,1400,1600,1900,2400,3000,3400,3800,4300,4800,5900,6300,7300,8500], 
        'color': '#2CA02C'
    },
    'Deadly': {
        'XP': [100,200,400,500,1100,1400,1700,2100,2400,2800,3600,4500,5100,5700,6400,7200,8800,9500,10900,12700], 
        'color': '#D62728'
    },
    'Daily':  {
        'XP': [300,600,1200,1700,3500,4000,5000,6000,7500,9000,10500,11500,13500,15000,18000,20000,25000,27000,30000,40000], 
        'color': '#9467BD'
    },
}

In [2]:
# Setup template and html config for plotly figures.

TFB_TEMPLATE = dict(
    layout=go.Layout(
        template='plotly_white',
        autosize=False,
        margin=dict(l=50, r=25, b=40, t=20, pad=4),
        font=dict(family='sans-serif', size=14),
        hovermode='closest',
        xaxis=dict(
            autorange=False,
            automargin=True,
            fixedrange=True,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear', ticks='outside',
            minor=dict(tickmode='linear', ticks='outside'),
            zeroline=False,
        ),
        yaxis=dict(
            autorange=False,
            automargin=True,
            fixedrange=True,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear', ticks='outside',
            minor=dict(tickmode='linear', ticks='outside'),
            zeroline=False,
        ),
        legend=dict(
            font_size=10,
            tracegroupgap=0,
            orientation='h',
            yanchor='bottom',  y=1.04,
            xanchor='left',    x=0.01,
            bgcolor='rgba(0,0,0,0)',
        ),
    )
)

TFB_CONFIG = {
    'responsive': True, # must be True to auto-scale when resizing
    #'autosizable': False, # doesn't impact auto rescaling
    'showAxisDragHandles': False,
    'displaylogo': False,
    'displayModeBar': 'hover',
    'modeBarButtonsToRemove': [
        'select2d',
        'lasso2d',
        'zoom2d',
        'zoomIn2d',
        'zoomOut2d',
        'pan2d',
        'autoScale2d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'toggleSpikelines',
        'resetScale2d',
    ],
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'tfb-plot',
        'height': 450,
        'width': 600,
        'scale': 2
    },
}

def save_fig_html(fig, format, name, **kwargs):
    file_name = f'./{name}.html'

    match format:
        case 'large':
            fig.update_traces(line_width=2, marker_size=8)
            fig.update_shapes(line_width=2)
            fig.update_layout(font_size=14)
            fig.update_annotations(font_size=14)
        case 'small':
            fig.update_traces(line_width=1, marker_size=6)
            fig.update_shapes(line_width=1)
            fig.update_layout(font_size=10, legend=dict(font=dict(size=10)))
            fig.update_annotations(font_size=10)

    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = f'plotly-div-{format}'
    if kwargs.get('style', None):
        fig_soup.div['style'] = kwargs.get('style')
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [3]:
# Encounter XP - BL
import copy

difficulty = 'Medium'
short_rests = 2
show_level_times = False

PC_CLASSES = {
    'Barbarian': {'color': '#E7623E', 'group': 'martial'},
    'Bard':      {'color': '#AB6DAC', 'group': 'spellcaster'},
    'Cleric':    {'color': '#91A1B2', 'group': 'spellcaster'},
    'Druid':     {'color': '#7A853B', 'group': 'spellcaster',},
    'Fighter':   {'color': '#7F513E', 'group': 'martial'},
    'Monk':      {'color': '#51A5C5', 'group': 'martial'},
    'Paladin':   {'color': '#B59E54', 'group': 'martial-spellcaster'},
    'Ranger':    {'color': '#507F62', 'group': 'martial-spellcaster'},
    'Rogue':     {'color': '#555752', 'group': 'martial'},
    'Sorcerer':  {'color': '#992E2E', 'group': 'spellcaster'},
    'Warlock':   {'color': '#7B469B', 'group': 'spellcaster'},
    'Wizard':    {'color': '#2A50A1', 'group': 'spellcaster'}
}

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

pcData = {}
class_time = []
for c in PC_CLASSES:
    levels = np.arange(1, 21, 1)
    print(f'{c:<10s}', end='')
    daily_engine_options={
        'combat healing': False if c == 'Paladin' else True,
        'remove empty data': True,
        'simplify summary': True,
        'effective_method': 'linear',
    }
    level_time = []
    pcData[c] = []
    for level in levels:
        tic = time.perf_counter()
        pc = pc5.newPC(pcClass=c, lvl=level, **PC_CLASSES[c])
        advDay = en5.create_adventuring_day(level, difficulty, short_rests)
        dSum = en5.dailyEngine(pc, advDay, **daily_engine_options)
        dSum['class'] = c
        dSum['level'] = int(level)
        pcData[c].append(copy.deepcopy(dSum))

        toc = time.perf_counter()
        level_time += [toc - tic]
        if show_level_times:
            print(f' {level}: {level_time[-1]:0.2f} s,', end='')
        else:
            print('.', end='')
        time.sleep(0.1)

    class_time += [np.sum(level_time)]
    print(f' total: {class_time[-1]:0.2f} s')
    time.sleep(0.1)

if SAVEDATA:
    # construct day string for saving figures
    file_name = 'pc data - BL {:} {:d} short rests.json'.format(difficulty, short_rests)
    with open(file_name, 'w') as convert_file:
        convert_file.write(json.dumps(pcData, indent=2, cls=NpEncoder))
    print('pc data saved to "{:}"'.format(file_name))

print(f'total run time: {np.sum(class_time):0.2f} s')

Barbarian .................... total: 0.78 s
Bard      .................... total: 1.35 s
Cleric    .................... total: 0.82 s
Druid     .................... total: 0.86 s
Fighter   .................... total: 0.85 s
Monk      .................... total: 1.12 s
Paladin   .................... total: 12.08 s
Ranger    .................... total: 1.04 s
Rogue     .................... total: 0.36 s
Sorcerer  .................... total: 4.62 s
Warlock   .................... total: 0.71 s
Wizard    .................... total: 9.00 s
total run time: 33.61 s


In [29]:
# Plots XP increase - +1 Attack Bonus
pc_data_file = f'./pc data - BL Medium 2 short rests.json'
with open(pc_data_file, 'r') as fin:
    pc_data0 = json.load(fin)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            fixedrange=False,
            #title_text='$(\AB + \AC - 2)^{-1}\ (\%)$',
            title_text='encounter XP increase (%)',
            range=[0, 8],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
    )
)

# plot data
levels = np.array(range(1,21,1))
ymean = np.zeros(20)
for pc in pc_data0:
    y = np.array([1/(x['armor class'] + x['attack bonus'] - 2) for x in pc_data0[pc]])
    ymean += y
    fig.add_trace(go.Scatter(
        x=levels, 
        y=100*y,
        mode='lines+markers', 
        line={'color': PC_CLASSES[pc]['color']},
        unselected=dict(marker=dict(opacity=0.0)),
        name=pc,
        hovertemplate=
            '<b>'+pc+'</b><br>'+
            'level %{x}<br>'+
            'XP increase %{y:.1f}%'+
            '<extra></extra>'
    ))

ymean /= len(pc_data0)
fig.add_trace(go.Scatter(
    x=levels, 
    y=100*ymean,
    mode='lines+markers', 
    line={'color': 'black', 'dash': 'dash'},
    unselected=dict(marker=dict(opacity=0.0)),
    name='average',
    hovertemplate=
        '<b>Average</b><br>'+
        'level %{x}<br>'+
        'XP increase %{y:.1f}%'+
        '<extra></extra>'
))
print(f'{np.mean(ymean):.2%}')

# show figure
fig.update_layout(width=600, height=550)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name=f'./fig-encounter-xp-generic-bonus-large', style='aspect-ratio:600/550')
    save_fig_html(fig, format='small', name=f'./fig-encounter-xp-generic-bonus-small', style='aspect-ratio:600/600')
    pass

4.16%


In [6]:
# Encounter XP - +1 Weapon Damage Bonus
import copy

difficulty = 'Medium'
short_rests = 2
show_level_times = False

PC_CLASSES = {
    'Barbarian': {'color': '#E7623E', 'group': 'martial',             'weaponDamageBonus': '+1'},
    #'Bard':      {'color': '#AB6DAC', 'group': 'spellcaster',         'weaponDamageBonus': '+1'},
    #'Cleric':    {'color': '#91A1B2', 'group': 'spellcaster',         'weaponDamageBonus': '+1'},
    #'Druid':     {'color': '#7A853B', 'group': 'spellcaster',         'weaponDamageBonus': '+1'},
    'Fighter':   {'color': '#7F513E', 'group': 'martial',             'weaponDamageBonus': '+1'},
    'Monk':      {'color': '#51A5C5', 'group': 'martial',             'weaponDamageBonus': '+1'},
    'Paladin':   {'color': '#B59E54', 'group': 'martial-spellcaster', 'weaponDamageBonus': '+1'},
    'Ranger':    {'color': '#507F62', 'group': 'martial-spellcaster', 'weaponDamageBonus': '+1'},
    'Rogue':     {'color': '#555752', 'group': 'martial',             'weaponDamageBonus': '+1'},
    #'Sorcerer':  {'color': '#992E2E', 'group': 'spellcaster',         'weaponDamageBonus': '+1'},
    #'Warlock':   {'color': '#7B469B', 'group': 'spellcaster',         'weaponDamageBonus': '+1'},
    #'Wizard':    {'color': '#2A50A1', 'group': 'spellcaster',         'weaponDamageBonus': '+1'}
}

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

pcData = {}
class_time = []
for c in PC_CLASSES:
    levels = np.arange(1, 21, 1)
    print(f'{c:<10s}', end='')
    daily_engine_options={
        'combat healing': False if c == 'Paladin' else True,
        'remove empty data': True,
        'simplify summary': True,
        'effective_method': 'linear',
    }
    level_time = []
    pcData[c] = []
    for level in levels:
        tic = time.perf_counter()
        pc = pc5.newPC(pcClass=c, lvl=level, **PC_CLASSES[c])
        advDay = en5.create_adventuring_day(level, difficulty, short_rests)
        dSum = en5.dailyEngine(pc, advDay, **daily_engine_options)
        dSum['class'] = c
        dSum['level'] = int(level)
        pcData[c].append(copy.deepcopy(dSum))

        toc = time.perf_counter()
        level_time += [toc - tic]
        if show_level_times:
            print(f' {level}: {level_time[-1]:0.2f} s,', end='')
        else:
            print('.', end='')
        time.sleep(0.1)

    class_time += [np.sum(level_time)]
    print(f' total: {class_time[-1]:0.2f} s')
    time.sleep(0.1)

if SAVEDATA:
    # construct day string for saving figures
    file_name = 'pc data - WD +1 {:} {:d} short rests.json'.format(difficulty, short_rests)
    with open(file_name, 'w') as convert_file:
        convert_file.write(json.dumps(pcData, indent=2, cls=NpEncoder))
    print('pc data saved to "{:}"'.format(file_name))

print(f'total run time: {np.sum(class_time):0.2f} s')

Barbarian .................... total: 0.79 s
Fighter   .................... total: 0.91 s
Monk      .................... total: 1.16 s
Paladin   .................... total: 12.17 s
Ranger    .................... total: 1.05 s
Rogue     .................... total: 0.40 s
pc data saved to "pc data - WD +1 Medium 2 short rests.json"
total run time: 16.48 s


In [25]:
# Plots XP increase - +1 Weapon Damage Bonus
encounter_difficulty = 'Medium'
pc_data_file = f'./pc data - BL {encounter_difficulty} 2 short rests.json'
with open(pc_data_file, 'r') as fin:
    pc_data0 = json.load(fin)

pc_data_file = f'./pc data - WD +1 {encounter_difficulty} 2 short rests.json'
with open(pc_data_file, 'r') as fin:
    pc_data1 = json.load(fin)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            fixedrange=False,
            title_text='encounter XP increase (%)',
            range=[0, 15],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
    )
)

# plot data
levels = np.array(range(1,21,1))
eXP0_mean = np.zeros(20)
eXP1_mean = np.zeros(20)
for pc in pc_data1:
    eXP0 = np.array([x['encounter XP mean'] for x in pc_data0[pc]])
    eXP1 = np.array([x['encounter XP mean'] for x in pc_data1[pc]])
    eXP0_mean += eXP0
    eXP1_mean += eXP1

    fig.add_trace(go.Scatter(
        x=levels, 
        #y=100*(eDPR1/eDPR0 - 1),
        y=100*(eXP1/eXP0 - 1),
        mode='lines+markers', 
        line={'color': PC_CLASSES[pc]['color']},
        unselected=dict(marker=dict(opacity=0.0)),
        name=pc,
        hovertemplate=
            '<b>'+pc+'</b><br>'+
            'level %{x}<br>'+
            'XP increase %{y:.3f}'+
            '<extra></extra>'
    ))

fig.add_trace(go.Scatter(
    x=levels, 
    y=100*(eXP1_mean/eXP0_mean - 1),
    mode='lines+markers', 
    line={'color': 'black', 'dash': 'dash'},
    unselected=dict(marker=dict(opacity=0.0)),
    name=pc,
    hovertemplate=
        '<b>Average</b><br>'+
        'level %{x}<br>'+
        'XP increase %{y:.1f}'+
        '<extra></extra>'
))

print(np.mean(eXP1_mean/eXP0_mean))

# show figure
fig.update_layout(width=600, height=550)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name=f'./fig-encounter-xp-damage-bonus-large', style='aspect-ratio:600/550')
    save_fig_html(fig, format='small', name=f'./fig-encounter-xp-damage-bonus-small', style='aspect-ratio:600/550')

1.0677026398278733


In [8]:
# Encounter XP - high magic
import copy

difficulty = 'Medium'
short_rests = 2
show_level_times = False

PC_CLASSES = {
    'Barbarian': {'color': '#E7623E', 'group': 'martial'},
    'Bard':      {'color': '#AB6DAC', 'group': 'spellcaster'},
    'Cleric':    {'color': '#91A1B2', 'group': 'spellcaster'},
    'Druid':     {'color': '#7A853B', 'group': 'spellcaster'},
    'Fighter':   {'color': '#7F513E', 'group': 'martial'},
    'Monk':      {'color': '#51A5C5', 'group': 'martial'},
    'Paladin':   {'color': '#B59E54', 'group': 'martial-spellcaster'},
    'Ranger':    {'color': '#507F62', 'group': 'martial-spellcaster'},
    'Rogue':     {'color': '#555752', 'group': 'martial'},
    'Sorcerer':  {'color': '#992E2E', 'group': 'spellcaster'},
    'Warlock':   {'color': '#7B469B', 'group': 'spellcaster'},
    'Wizard':    {'color': '#2A50A1', 'group': 'spellcaster'}
}

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

pcData = {}
class_time = []
for c in PC_CLASSES:
    levels = np.arange(1, 21, 1)
    print(f'{c:<10s}', end='')
    daily_engine_options={
        'combat healing': False if c == 'Paladin' else True,
        'remove empty data': True,
        'simplify summary': True,
        'effective_method': 'linear',
    }
    level_time = []
    pcData[c] = []
    for level in levels:
        # determine magic item bonus
        bonus = 0
        if level > 4:
            bonus += 1
        if level > 9:
            bonus += 1
        if level > 15:
            bonus += 1
        pc_options = {
            'armorClassBonus': bonus,
            'spellAttackBonus': bonus,
            'weaponAttackBonus': bonus,
            'weaponDamageBonus': f'+{bonus}',
        }

        tic = time.perf_counter()
        pc = pc5.newPC(pcClass=c, lvl=level, **pc_options)
        advDay = en5.create_adventuring_day(level, difficulty, short_rests)
        dSum = en5.dailyEngine(pc, advDay, **daily_engine_options)
        dSum['class'] = c
        dSum['level'] = int(level)
        pcData[c].append(copy.deepcopy(dSum))

        toc = time.perf_counter()
        level_time += [toc - tic]
        if show_level_times:
            print(f' {level}: {level_time[-1]:0.2f} s,', end='')
        else:
            print('.', end='')
        time.sleep(0.1)

    class_time += [np.sum(level_time)]
    print(f' total: {class_time[-1]:0.2f} s')
    time.sleep(0.1)

if SAVEDATA:
    # construct day string for saving figures
    file_name = 'pc data - high magic {:} {:d} short rests.json'.format(difficulty, short_rests)
    with open(file_name, 'w') as convert_file:
        convert_file.write(json.dumps(pcData, indent=2, cls=NpEncoder))
    print('pc data saved to "{:}"'.format(file_name))

print(f'total run time: {np.sum(class_time):0.2f} s')

Barbarian .................... total: 0.73 s
Bard      .................... total: 1.37 s
Cleric    .................... total: 0.83 s
Druid     .................... total: 0.81 s
Fighter   .................... total: 0.85 s
Monk      .................... total: 1.12 s
Paladin   .................... total: 12.00 s
Ranger    .................... total: 1.03 s
Rogue     .................... total: 0.37 s
Sorcerer  .................... total: 4.55 s
Warlock   .................... total: 0.71 s
Wizard    .................... total: 9.08 s
pc data saved to "pc data - high magic Medium 2 short rests.json"
total run time: 33.45 s


In [30]:
# Plots XP increase - high magic
encounter_difficulty = 'Medium'
pc_data_file = f'./pc data - BL {encounter_difficulty} 2 short rests.json'
with open(pc_data_file, 'r') as fin:
    pc_data0 = json.load(fin)

pc_data_file = f'./pc data - high magic {encounter_difficulty} 2 short rests.json'
with open(pc_data_file, 'r') as fin:
    pc_data1 = json.load(fin)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            fixedrange=False,
            title_text='encounter XP increase (%)',
            range=[-1, 50],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=5),
        ),
    )
)

# plot data
levels = np.array(range(1,21,1))
eXP0mean = np.zeros(20)
eXP1mean = np.zeros(20)
for pc in PC_CLASSES:
    eXP0 = np.array([x['encounter XP mean'] for x in pc_data0[pc]])
    eXP0mean += eXP0
    eXP1 = np.array([x['encounter XP mean'] for x in pc_data1[pc]])
    eXP1mean += eXP1

    fig.add_trace(go.Scatter(
        x=levels, 
        y=100*(eXP1/eXP0 - 1),
        mode='lines+markers', 
        line={'color': PC_CLASSES[pc]['color']},
        unselected=dict(marker=dict(opacity=0.0)),
        name=pc,
        hovertemplate=
            '<b>'+pc+'</b><br>'+
            'level %{x}<br>'+
            'XP increase %{y:.1f}'+
            '<extra></extra>'
    ))

fig.add_trace(go.Scatter(
    x=levels, 
    y=100*(eXP1mean/eXP0mean - 1),
    mode='lines+markers', 
    line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='average',
    hovertemplate=
        '<b>Average</b><br>'+
        'level %{x}<br>'+
        'XP increase %{y:.1f}'+
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=550)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name=f'./fig-encounter-xp-high-magic-large', style='aspect-ratio:600/550')
    save_fig_html(fig, format='small', name=f'./fig-encounter-xp-high-magic-small', style='aspect-ratio:600/600')